## Step 1: Using SeqIO, read in and parse the file of apple primary transcripts (available on Canvas):
    - During your parsing, create a list of only transcripts of length 125 or less
    - Report the number of transcripts matching this criteria
    - Translate these sequences to protein, make sure to save them to their own list

In [229]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna


sequences = []
translated_sequences = []
coding_dna = ""

# Filters sequences by length. Add sequence to list if 125 bases or fewer
for record in SeqIO.parse("Mdomestica_491_v1.1.cds_primaryTranscriptOnly.fa", "fasta"):
        if len(record.seq) <= 125:
            sequences.append(record.seq)
print("Number of sequences that are 125 bases or less: " + str(len(sequences)))
    

# Iterate through sequences list. Translate each sequence element and append to new list
for x in sequences:
    coding_dna = Seq(str(x), generic_dna)
    translated_sequences.append(coding_dna.translate())

# Prints new list of translated sequences   
for x in translated_sequences:
    
    print(x)

    

    





Number of sequences that are 125 bases or less: 62
MYGAGAKEDLQLCSGCLCTCSAEALQIQRKRRRVYNFLCT*
MGPYRSQQIKSQPNGHVYGQRGHDHRRDEVVAAPGAPDRV*
MILGYHRPLNGLEVGRTVAVRGAVVDGAWSTGASTGRMIA*
MPLSAHGTSEIFSKNMGMFVRLCRSRWYIQTSHSSNLFRI*
MVKYSREPDNITKSCKARGSDLRVHFKVFGLFKFGQFGQD*
MVGVPQVFRRGELPNEVSCLLLICQSNESLFRSQIAKSSS*
MIRRSKYCERHAKKEHITNTTPDEERDENISDEESKLKGQ*
MPVVLRELDLESSCMVKFTVDLLCYELRSSWIVFRPQTRK*
MGSIDFVERGTGSVVFGRAMRKRIEWCFRLVLQMSYGLSN*
MDQITSFLAHVHPKVHPGLVTGRYIWLLLIVTAACVIKQH*
MIEMFVNRKIVNLIGRLGIELELEVSHVSWSDAVPNILLS*
MLMNGFNEAKRETDMASSTSSGAAGNFYTWNGRIVLCAFF*
MGPTRKDWSLRLDDALWAYRTAYKTPIGMSPFRLIYAPIK*
MKLFWVKFWRCQSLEVLVMRALLALALSLSLQCRSLQGLC*
MIEAKNQALFAGHLVELELADGYFECAGLVSLVHAAYACR*
MTKHCPQIDSSTQVPDLSCRPCGKESCESNLHVSVLCPEG*
MKKPDSAVKRDRKAEKRRIYNKSQKSEIKTRMKKVVMSFH*
MSAELAEVGLELMAAEVEEAGLKLTSAETGLNLGCAAEAG*
MLSDSEENDHDQTEQLMIADSNVDVTIARLSTKKESSSFA*
MLIQLRKSLRNTASKPVCGRLFLGPGYEENTYAQWEWAID*
MVCTKRVADWESWRAIPKELKMHMIDELAPIGTLTKVAQI*
MGYLGCASPPIFETLVLLRSVCDPPEKPRETPSCCLARKN*
MQPDTGQGEGDEDEDVDKNDIEKHN

## Step 2: Align our small proteins using pairwise2
    - Align each protein to each other protein using the blosum62 scoring matrix, a gap opening penalty of -10, and a gap extension penalty of -0.5
    - If an alignment scores better than 40, save it.
    - Print the highest scoring pairwise alignment
    - Take care in how you set up your loops for the pairwise alignments. Each alignment only needs to be done once. If you've already tested the alignment of seq1 vs seq5, don't align seq5 vs seq1.
    

In [230]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import AlignIO
from Bio import Align
from Bio.SubsMat.MatrixInfo import blosum62

#matrix = matlist.blosum62

aligned_sequences = []
scored_sequences = []
aligner = Align.PairwiseAligner()


# Iterate through list of proteins and only align unique combinations of seqA and seqB
for i in range(0, len(translated_sequences)-1, 1):
    for j in range(i+1, len(translated_sequences), 1):
        
        # for each alignment object, use BLOSUM62 scoring matrix to score alignment
        for a in pairwise2.align.globalds(translated_sequences[i][0:-1],translated_sequences[j][0:-1], blosum62, -10, -0.5):
            
            # only keep alignments whose score is above 40
            if a[2] > 40:
                scored_sequences.append(a)
            #print(format_alignment(*a))
            
        
for x in scored_sequences:
    print(x)
 
        





('MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL', 'MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL', 211.0, 0, 40)


## Step 3: Running BLAST and reading results
     *Because we are doing a web BLAST, choose ONLY one sequence from our list of short proteins*.
    - Run BLAST with your sequence against the NR database
    - Parse the results. Report any HSPs with an E-value less than 0.05 and show the HSP alignments, including the name of the matching sequence. If no HSPs meet that criteria, report the highest scoring pair.
    - If for whatever reason the sequence you selected fails to return any results, try a new one 

In [231]:
from Bio import SearchIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML


#Blast protein sequence and create a BLAST results object (result_handle) 
result_handle = NCBIWWW.qblast('blastp', 'nr', 'MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL')

#Open new file to write blast resutlts object 
with open("/mnt/c/Users/adamh/OneDrive/Desktop/blast_results.xml", 'w') as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()

#blast_out = open('/mnt/c/Users/adamh/OneDrive/Desktop/blast_results.xml', 'r')





KeyboardInterrupt: 

In [ ]:
from Bio.Blast import NCBIXML

# parse Blast results object 
blast_record = NCBIXML.parse(result_handle)

E_VALUE_THRESH = 0.05

#iterate through object and store sequences and scores below threshold
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("Sequence: " + alignment.title)
            print("E-value: " + hsp.expect)
            #print("Highest scoring: " + max())
           